In [42]:
import pickle
import pandas as pd
import re
import numpy as np
import datetime


from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification

from nltk.tokenize import word_tokenize
import torch

In [3]:
brand = 'maxima'

In [4]:
def cleanMessage(df):
    newLine ="\\n|\\r"
    urls = '(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})'
    numbers = '\d+((\.|\-)\d+)?'
    mentions = '\B\@([\w\-]+)'
    hashtag = '#'
    whitespaces = '\s+'
    leadTrailWhitespace = '^\s+|\s+?$'

    df['clean_message'] = df['message']
    df['clean_message'] = df['clean_message'].str.replace(newLine,' ',regex=True)
    df['clean_message'] = df['clean_message'].str.replace(urls,' URL ',regex=True)
    df['clean_message'] = df['clean_message'].str.replace(mentions,' MENTION ',regex=True)
    df['clean_message'] = df['clean_message'].str.replace(numbers,' NMBR ',regex=True)
    df['clean_message'] = df['clean_message'].str.replace(hashtag,' ',regex=True)
    df['clean_message'] = df['clean_message'].str.replace(whitespaces,' ',regex=True)
    df['clean_message'] = df['clean_message'].str.replace(leadTrailWhitespace,'',regex=True)

In [32]:
def readCsv(brand):
    tweets = pd.read_csv('./../peopleTweets/'+brand+'.csv')
    tweets['brand'] = brand
    for index, row in brandTweets.iterrows():
#         if not(np.isnan(row['label'])):
#             row['label'] = int(row['label'])
    cleanMessage(tweets)
    return tweets

In [33]:
brandTweets = readCsv(brand)
brandTweets.head()

,message,id,tweetId,createdAt,language,inReplyToStatusId,inReplyToUserId,inReplyToScreenName,userId,userName,...,placeType,retweetedId,monitoringObjectId1,monitoringObjectId2,queryId,sentiment,retweetCount,label,brand,clean_message
0,@VienotibaLV @EM_gov_lv Kad sabruka Maxima atk...,1479501,925656400235237400,2017-11-01T11:30:58,lv,9.253513e+17,163366515.0,VienotibaLV,912753641114697700,Chiulju Pussala,...,NaN,NaN,684,684,923.0,NaN,0,NaN,maxima,MENTION MENTION Kad sabruka Maxima atkāpās pre...
1,"Kļuvis zināms, kādēļ Latvijā krieviski runā un...",1479898,925246907835912200,2017-10-31T08:23:47,lv,NaN,NaN,NaN,75507864,Māris Luste,...,NaN,NaN,613,3,NaN,-0.628,0,2.0,maxima,"Kļuvis zināms, kādēļ Latvijā krieviski runā un..."
2,@DaceKalnina Un kāpēc portālā virs šī raksta e...,1494034,926042411029205000,2017-11-02T13:04:50,lv,9.256348e+17,26082263.0,DaceKalnina,131121745,Jānis Goldbergs,...,NaN,NaN,230,230,469.0,NaN,0,NaN,maxima,MENTION Un kāpēc portālā virs šī raksta es vie...
3,🎉🍸😏😏🙏🏽 (@ Maxima XX - @maximaveikals in Tukums...,1519965,926808202510983200,2017-11-04T15:47:49,sw,NaN,NaN,NaN,2729790340,Lelde,...,city,NaN,199,199,438.0,NaN,0,NaN,maxima,"🎉🍸😏😏🙏🏽 (@ Maxima XX - MENTION in Tukums, Tukum..."
4,"@GirtsRungainis Skarbākais, ka vecāku absolūta...",1521601,927041580455874600,2017-11-05T07:15:11,lv,9.268910e+17,134731218.0,GirtsRungainis,701302008,NK,...,NaN,NaN,269,269,508.0,NaN,0,2.0,maxima,"MENTION Skarbākais, ka vecāku absolūtais vairu..."


In [37]:
def convertDates(df):
    dates = []
    for index, row in df.iterrows():
        date_no_0 = row['createdAt'].replace('+00:00','')
        dates.append(datetime.datetime.strptime(date_no_0, '%Y-%m-%dT%H:%M:%S').strftime('%Y,%m,%d'))

    df["date"] = dates

In [38]:
convertDates(brandTweets)

In [43]:
# Tokenizer

tokenizer = BertTokenizer.from_pretrained('./lvBERT/lvbert_pytorch/', do_lower_case=False)

In [45]:
unk_tokens = [('^vi', 34), ('^ai', 30), ('^el', 22), ('(', 19), ('^il', 19), ('–', 17), ('^la', 17), ('^na', 17), ('€', 15), ('^ma', 15), ('^ri', 13), ('+', 11), ('X', 11), (':(', 8), ('Ļoti', 8), ('^ju', 8), ('🇱🇻', 7), ('&amp;', 6), ('👉', 6), ('🤔', 5), ('MAXIMA', 5), ('🎄', 5), ('€.', 4), ('😉', 4), ('^nn', 4), ('😅', 3), ('|', 3), ('(vai', 3), ('😊', 3), ('ibankā', 3), ('Žēl,', 3), ('You', 3), ('не', 3), ('за', 3), ('в', 3), ('❤️', 3), ('▶️', 3), ('uzs…', 2), ('YL-CSK.', 2), ('(un', 2), ('nav…', 2), ('😍', 2), ('(pat', 2), ('=', 2), ('🤗', 2), ('ļoooti', 2), ('(kaut', 2), ('XX', 2), ('“Valmiera/ORDO”', 2), ('😀', 2), ('🙄', 2), ('~', 2), ('🇷🇺', 2), ('(@', 2), ('*', 2), ('🤷\u200d♀️', 2), ('🤬', 2), ('✨', 2), ('(FKTK)', 2), ('(VDD)', 2), ('(sauksim', 2), ('Если', 2), ('🙂', 2), ('Yes,', 2), ('(+', 2), ('нас', 2), ('Добрый', 2), ('день!', 2), ('Pārbaudīsim.^la', 2), ('iOS', 2), ('“Maxima', 2), ('[..]', 2), ('XXX', 2), ('📲', 2), ('“potenciālie', 1), ('pasaules”', 1), ('(maijā),', 1), ('“', 1), ('slīpie”', 1), ('*FB', 1), ('konkursu*', 1), ('Pa…', 1), ('liepajatravel…', 1), ('✈️Izdevīgas', 1), ('laicīgi👍', 1), ('Qatar', 1), ('QR', 1), ('lidmašīnas😀😀😀', 1), ('(Rīgā:', 1), ('Havaś)', 1), ('Ņefiga', 1), ('👍🤗', 1), ('😥😥', 1), ('😵', 1), ('🇫🇷un', 1), ('krāšņumā…', 1), ('IONITY', 1), ('Ļooti', 1), ('skočs…', 1), ('nava😖', 1)]
most_common_values= [word for word, word_count in unk_tokens]

tokenizer.add_tokens(most_common_values, special_tokens=True)

100

In [46]:
# Find max length for tokenizer

token_lens = []
for txt in list(brandTweets.clean_message.values):
    tokens = tokenizer.encode(txt, max_length=512, truncation=True)
    token_lens.append(len(tokens))
    
max_length = max(token_lens)
max_length

185

In [48]:
import torch

In [49]:
# Encode messages

# encoded_data = tokenizer.batch_encode_plus(
#     brandTweets["clean_message"].values, 
#     add_special_tokens=True, 
#     return_attention_mask=True, 
#     padding='max_length',
#     truncation=True,
#     max_length=max_length, 
#     return_tensors='pt'
# )

# input_ids = encoded_data['input_ids']
# attention_masks = encoded_data['attention_mask']
# labels = torch.tensor(brandTweets.label.values)

# dataset = TensorDataset(input_ids, attention_masks, labels)

# len(dataset)

5361

In [50]:
# Model "lvBERT"

model = BertForSequenceClassification.from_pretrained('./lvBERT/lvbert_pytorch/',
                                                      num_labels=3,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at ./lvBERT/lvbert_pytorch/ were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkp

In [51]:
model.resize_token_embeddings(len(tokenizer))

Embedding(32104, 768)

In [52]:
# Load best model

model.load_state_dict(torch.load('./lvBERT/addUNKtokens/modelsUNK/finetuned_lvBERT_epoch_1.model', map_location=torch.device('cpu')))

<All keys matched successfully>

In [55]:
def predict(model, data):
    token_lens = []
    for txt in list(data["clean_message"].values):
        tokens = tokenizer.encode(txt, max_length=512, truncation=True)
        token_lens.append(len(tokens))
    max_length = max(token_lens)

    encoded_data = tokenizer.batch_encode_plus(
        data["clean_message"].values, 
        add_special_tokens=True, 
        return_attention_mask=True, 
        padding='max_length',
        truncation=True,
        max_length=max_length, 
        return_tensors='pt'
    )

    input_ids = encoded_data['input_ids']
    attention_masks = encoded_data['attention_mask']
    labels = torch.tensor(data.label.values)
    
    outputs = model(**encoded_data)
    logits = outputs.logits
    logits = logits.detach().cpu().numpy()
    
    predictions = np.argmax(logits, axis=1).flatten()
    return predictions, labels

In [56]:
predictions, labels = predict(model, brandTweets)
print(predictions)
print(labels)

RuntimeError: [enforce fail at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\core\impl\alloc_cpu.cpp:81] data. DefaultCPUAllocator: not enough memory: you tried to allocate 12187054080 bytes.